In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import CameraNetwork
import CameraNetwork.global_settings as gs
import cv2
import Image
import ImageDraw
import time
from tqdm import tqdm_notebook

c:\users\user\documents\code\cameranetwork_git\CameraNetwork\transformation_matrices.py:1899: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


In [4]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
SERVERS = c.client_instance.servers
print SERVERS

['109', '110', '111', '112', '113', '114', '115', '116', '117', '118']


In [5]:
df = c.query('117', '03/13/2017')

In [6]:
video = cv2.VideoWriter('video_117_2017_03_13.avi', -1, 6, (301, 301))

for ind in tqdm_notebook(df.loc[(slice(None), '2'), :].index):
    img, _ = c.seek('117', ind[0], ind[1], True, 301)
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    draw.text(
        (0, 0),
        ind[0].time().strftime("%H:%M:%S"),
        (255,255,255)
    )

    video.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
                             
video.release()

In [10]:
df.shape

(5, 1)